# IEOR 142 HW4 Starter Code — Fall 2023

In [22]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import sklearn.datasets
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import statsmodels.formula.api as smf
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier

In [23]:
# Load in data
train = pd.read_csv('Letters_train.csv', index_col = 0)
test = pd.read_csv('Letters_test.csv', index_col = 0)
# TODO: Load in data (after analyzing the dataset, delete any ouputs such as df.inf(), df.head(), et).
# this cell should not output anything

# Question 2 (25 points)

In [24]:
# TODO: Create new variable here
train['isB'] = train['letter'].map(lambda x: 'Yes' if x == 'B' else 'No')
test['isB'] = test['letter'].map(lambda x: 'Yes' if x == 'B' else 'No')

In [25]:
# TODO: Split into X and y

### Part A: Baseline Model (3 points)

In [26]:
# Q1A code
y_test = test['isB']
baseline_pred = np.repeat('No', len(test))
cm1 = confusion_matrix(baseline_pred, y_test)
TN = cm1.ravel()[0]
FP = cm1.ravel()[1]
FN = cm1.ravel()[2]
TP = cm1.ravel()[3]
accuracy = (TP + TN) / (TP + TN + FN + FP)
#TODO: find the most common label

baseline_1_acc = accuracy # TODO: calcuate baseline accuracy
print(f'Baseline Test Accuracy: {baseline_1_acc:.4f}')

Baseline Test Accuracy: 0.7743


### Part B: Logistic Regression (5 points)

In [28]:
# Q1B code
letters_train_dummies = pd.get_dummies(train, drop_first = True)
letters_test_dummies = pd.get_dummies(test, drop_first = True)
letters_train_dummies = letters_train_dummies.rename(columns = {'isB_Yes' : 'isB'})
letters_test_dummies = letters_test_dummies.rename(columns = {'isB_Yes' : 'isB'})
letters_train_dummies = letters_train_dummies.drop(columns = {'letter_B', 'letter_P', 'letter_R'})
letters_test_dummies = letters_test_dummies.drop(columns = {'letter_B', 'letter_P', 'letter_R'})


logreg1 = smf.logit(formula = 'isB ~ xbox + ybox + width + height + onpix + xbar + ybar + x2bar + y2bar + xybar + x2ybar + xy2bar + xedge + xedgeycor + yedge + yedgexcor', data = letters_train_dummies).fit()
y_test2 = letters_test_dummies['isB']

log_mod_predictions = logreg1.predict(test)
log_mod_pred = pd.Series([1 if x > 0.5 else 0 for x in log_mod_predictions])

cm2 = confusion_matrix(log_mod_pred, y_test2)
TN2 = cm2.ravel()[0]
FP2 = cm2.ravel()[1]
FN2 = cm2.ravel()[2]
TP2 = cm2.ravel()[3]
log_model_acc = (TP2 + TN2) / (TP2 + TN2 + FN2 + FP2)
# TODO: For all questions: Create and train model, then make predictions, then calculate accuracy

model_1b_acc = log_model_acc
print(f'Logistic Regression Test Accuracy: {model_1b_acc:.4f}')

Optimization terminated successfully.
         Current function value: 0.159344
         Iterations 10
Logistic Regression Test Accuracy: 0.9401


### Part C: AUC (2 point)

In [35]:
# Q1C code
fpr, tpr, _ = roc_curve(y_test2, log_mod_pred)
model_1b_auc = auc(fpr, tpr)
print(f'Logistic Regression Test AUC: {model_1b_auc:.4f}')

Logistic Regression Test AUC: 0.9210


### Part D: Cross-validated CART (5 points)

**Written Answer**: In order to find the complexity parameter value that would optimize the accuracy of my model, I used GridSearchCV. First, I tested every value from np.linespace(0, 0.1, 200) and put the values in the dataframe to check for the best ccp_alpha that returns the best accuracy. I used ten folds for cross-validation because the dataset was big and the returned chart showed the ccp_alpha value of 0.001508 gave the best accuracy of 0.9294. However I also used .best_estimator_ to let it choose the best ccp_alpha realized it also chose a very similar value (practically same) of 0.0015. The accuracy returned from using 0.001508 was 0.9283 and the accuracy returned from using 0.0015 was 0.9305 so I concluded that ccp_alpha of 0.0015 returns the best accuracy of 0.9305.

In [36]:
letters_train_dummies = pd.get_dummies(train, drop_first = True)
letters_test_dummies = pd.get_dummies(test, drop_first = True)
letters_train_dummies = letters_train_dummies.rename(columns = {'isB_Yes' : 'isB'})
letters_test_dummies = letters_test_dummies.rename(columns = {'isB_Yes' : 'isB'})
letters_train_dummies = letters_train_dummies.drop(columns = {'letter_B', 'letter_P', 'letter_R'})
letters_test_dummies = letters_test_dummies.drop(columns = {'letter_B', 'letter_P', 'letter_R'})
x_train = letters_train_dummies.iloc[:, 0: 16]
x_test = letters_test_dummies.iloc[:, 0: 16]
y_train = letters_train_dummies['isB']
y_test = letters_test_dummies['isB']

In [37]:
dtc = DecisionTreeClassifier()
grid_values = {'ccp_alpha': np.linspace(0, 0.1, 200),
               'min_samples_leaf': [5],
               'min_samples_split': [20],
               'random_state': [2023]}
dtc_cv = GridSearchCV(dtc, param_grid = grid_values, scoring = 'accuracy', cv=10, verbose=0).fit(x_train, y_train)
acc = dtc_cv.cv_results_['mean_test_score']
ccp = dtc_cv.cv_results_['param_ccp_alpha'].data

In [38]:
pd.DataFrame({'ccp alpha' : ccp, 'Validation Accuracy': acc}).sort_values(by='Validation Accuracy', ascending=False).head(5)

,ccp alpha,Validation Accuracy
3,0.001508,0.929396
4,0.00201,0.928939
8,0.00402,0.928017
5,0.002513,0.926645
9,0.004523,0.926641


In [41]:
dtc_test = DecisionTreeClassifier(ccp_alpha=0.001508,
                              random_state = 2023)
dtc_test = dtc_test.fit(x_train, y_train)
y_pred_dtc_test = dtc_test.predict(x_test)
dec_tree_cm = confusion_matrix(y_test, y_pred_dtc_test)
TN3 = dec_tree_cm.ravel()[0]
FP3 = dec_tree_cm.ravel()[1]
FN3 = dec_tree_cm.ravel()[2]
TP3 = dec_tree_cm.ravel()[3]
accuracy = (TP3 + TN3) / (TP3 + TN3 + FN3 + FP3)
print(accuracy)

0.9283422459893048


In [42]:
# Q1D Code
dec_tree_pred = dtc_cv.best_estimator_.predict(x_test)
dec_tree_cm2 = confusion_matrix(y_test, dec_tree_pred)
TN3 = dec_tree_cm2.ravel()[0]
FP3 = dec_tree_cm2.ravel()[1]
FN3 = dec_tree_cm2.ravel()[2]
TP3 = dec_tree_cm2.ravel()[3]
accuracy = (TP3 + TN3) / (TP3 + TN3 + FN3 + FP3)
model_1d_acc = accuracy
model_1d_best_ccp_alpha = dtc_cv.best_params_['ccp_alpha']
print(f'CV CART Test Accuracy: {model_1d_acc:.4f}')
print(f'Best ccp_alpha: {model_1d_best_ccp_alpha:.4f}')

CV CART Test Accuracy: 0.9305
Best ccp_alpha: 0.0015


### Part E: Random Forest (5 points)


In [51]:
# Q1E Code
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
rf_pred = rf.predict(x_test)
rf_cm = confusion_matrix(y_test, rf_pred)
TN4 = rf_cm.ravel()[0]
FP4 = rf_cm.ravel()[1]
FN4 = rf_cm.ravel()[2]
TP4 = rf_cm.ravel()[3]
accuracy = (TP4 + TN4) / (TP4 + TN4 + FN4 + FP4)

model_1e_acc = accuracy
print(f'Random Forest Test Accuracy: {model_1e_acc:.4f}')

Random Forest Test Accuracy: 0.9850


### Part F: Performance Comparison (5 points)

**Written Answer**: The Logistic Model had an accuracy of 0.940, while the CART Model had an accuracy of .930 and the Random Forest Model had an accuracy around 0.985. The Random Forest Model had the highest accuracy and performed the best on the test set with the accuracy around 0.985. In this case, I believe that accuracy is more important than interpretability. There are a lot of variables in this dataset that are not interpretable, such as xbox, ybox, x2bar, yedgexor. Therefore, since we cannot interpret the variable’s impact on the accuracy, it would be harder to create a model that is interpretable and has a higher accuracy than what we already have. We should focus on predicting whether the letter is B and how accurate we can get our model.

In [53]:
# Q1F Code
print("Logistic Model Accuracy: ", model_1b_acc)
print('CART Model Accuracy: ', model_1d_acc)
print('Random Forest Model Accuracy: ', model_1e_acc)
# TODO: create df to compare performance/

Logistic Model Accuracy:  0.9401069518716577
CART Model Accuracy:  0.93048128342246
Random Forest Model Accuracy:  0.9850267379679144


***
# Question 3 (50 points)

In [54]:
# TODO: Redefine target y
train = pd.read_csv('Letters_train.csv', index_col = 0)
test = pd.read_csv('Letters_test.csv', index_col = 0)

### Part A: Baseline Model (5 points)

In [55]:
train.groupby('letter').size()

letter
A    553
B    555
P    575
R    498
dtype: int64

In [56]:
test.groupby('letter').size()

letter
A    236
B    211
P    228
R    260
dtype: int64

In [57]:
# Q2A
baseline_2_acc = (228) / (236 + 211+ 228 + 260)
print(f'Baseline Test Accuracy: {baseline_2_acc:.4f}')

Baseline Test Accuracy: 0.2439


### Part B: LDA (8 points)

In [59]:
# Q2B code
xtrain_lda = train.drop(columns = {'letter'})
xtest_lda = test.drop(columns = {'letter'})
ytrain_lda = train['letter']
ytest_lda = test['letter']

lda = LinearDiscriminantAnalysis()
fitted_lda = lda.fit(xtrain_lda, ytrain_lda)
lda_pred = fitted_lda.predict(xtest_lda)
lda_accuracy = sum(ytest_lda == lda_pred) / len(ytest_lda)
model_2b_acc = lda_accuracy
print(f'LDA Test Accuracy: {model_2b_acc:.4f}')

LDA Test Accuracy: 0.9102


### Part C: Cross-validated CART (10 points)

**Written Answer**: In order to find the complexity parameter value that would optimize the accuracy of my model, I used GridSearchCV. I tested every value from np.linespace(0, 0.1, 200) and this time, learning from the last time, I went straight to using .best_estimator_ in order to get the best accuracy and the best ccp_alpha. The best ccp_alpha value it chose was 0.0015 and the returned CART accuracy was 0.9198.

In [67]:
xtrain_cart = train.drop(columns = {'letter'})
xtest_cart = test.drop(columns = {'letter'})
ytrain_cart = train['letter']
ytest_cart = test['letter']

In [74]:
grid_values = {'ccp_alpha': np.linspace(0, 0.1, 200),
               'min_samples_leaf': [5],
               'min_samples_split': [20],
               'max_depth': [30],
               'random_state': [2023]}

dtc = DecisionTreeClassifier()
dtc_cv_acc2 = GridSearchCV(dtc, param_grid = grid_values, cv=10, verbose=1,
                              scoring = 'accuracy')
dtc_cv_acc2.fit(xtrain_cart, ytrain_cart)

Fitting 10 folds for each of 200 candidates, totalling 2000 fits


GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'ccp_alpha': array([0.        , 0.00050251, 0.00100503, 0.00150754, 0.00201005,
       0.00251256, 0.00301508, 0.00351759, 0.0040201 , 0.00452261,
       0.00502513, 0.00552764, 0.00603015, 0.00653266, 0.00703518,
       0.00753769, 0.0080402 , 0.00854271, 0.00904523, 0.00954774,
       0.01005025, 0.01055276, 0.01105528, 0.01155779, 0.0120603 ,
       0...
       0.0879397 , 0.08844221, 0.08894472, 0.08944724, 0.08994975,
       0.09045226, 0.09095477, 0.09145729, 0.0919598 , 0.09246231,
       0.09296482, 0.09346734, 0.09396985, 0.09447236, 0.09497487,
       0.09547739, 0.0959799 , 0.09648241, 0.09698492, 0.09748744,
       0.09798995, 0.09849246, 0.09899497, 0.09949749, 0.1       ]),
                         'max_depth': [30], 'min_samples_leaf': [5],
                         'min_samples_split': [20], 'random_state': [2023]},
             scoring='accuracy', verbose=1)

In [77]:
# Q2C Code
dec_tree_pred2 = dtc_cv_acc2.best_estimator_.predict(xtest_cart)
model_1d_best_ccp_alpha = dtc_cv_acc2.best_params_['ccp_alpha']
model_2c_acc = sum(dec_tree_pred2 == ytest_cart) / len(ytest_cart)
model_2c_best_ccp_alpha = dtc_cv_acc2.best_params_['ccp_alpha']
print(f'CART Test Accuracy: {model_2c_acc:.4f}')
print(f'Best ccp_alpha: {model_2c_best_ccp_alpha:.4f}')

CART Test Accuracy: 0.9198
Best ccp_alpha: 0.0015


### Part D: Vanilla Bagging (8 points)

In [78]:
# Q2D
xtrain_vb = train.drop(columns = {'letter'})
xtest_vb = test.drop(columns = {'letter'})
ytrain_vb = train['letter']
ytest_vb = test['letter']

rf2 = RandomForestClassifier(max_features=16, min_samples_leaf=5,n_estimators = 500, random_state=2023, verbose=0)
rf2.fit(xtrain_vb, ytrain_vb)
rf2_pred = rf2.predict(xtest_vb)
model_2d_acc = sum(rf2_pred == ytest_vb) / len(ytest_vb)
print(f'No CV Random Forest Test Accuracy: {model_2d_acc:.4f}')

No CV Random Forest Test Accuracy: 0.9422


### Part E: Cross-validated Random Forest (10 points)

**Written Answer**: In order to find the maximum features value that would optimize the accuracy of my model, I used GridSearchCV. I tested every value from np.linespace(1, 16, 16). I used ten folds for cross-validation because the dataset was big. First I set the n_estimators to around 400 but I realized that it was just running on and on forever. So I brought down the n_estimators value to 200 and it was able to finish running after a while. Using the best parameters that I found through cross-validation, I used the model to predict values and compared them to the test data. This gave a validation accuracy of 0.9658.

In [79]:
xtrain_crf = train.drop(columns = {'letter'})
xtest_crf = test.drop(columns = {'letter'})
ytrain_crf = train['letter']
ytest_crf = test['letter']

In [80]:
grid_values = {'max_features': np.linspace(1,16,16, dtype='int32'),
               'min_samples_leaf': [5],
               'n_estimators': [200],
               'random_state': [2023]}
rf3 = RandomForestClassifier()
rf_cv = GridSearchCV(rf3, param_grid=grid_values, scoring='accuracy', cv=10,verbose=0,n_jobs=-1).fit(xtrain_crf, ytrain_crf)

In [83]:
# Q2E
rf_crf_best_model = RandomForestClassifier(max_features = rf_cv.best_params_['max_features'], min_samples_leaf=5, n_estimators = 200, random_state=2023, verbose=0)
rf_crf_best_model.fit(xtrain_crf, ytrain_crf)
rf_crf_pred = rf_crf_best_model.predict(xtest_crf)
model_2e_acc = sum(rf_crf_pred == ytest_crf) / len(ytest_crf)
print(f'CV Random Forest Test Accuracy: {model_2e_acc:.4f}')

CV Random Forest Test Accuracy: 0.9658


### Part F: Gradient Boosting Classifier (9 points)

In [84]:
# Q2F
xtrain_gbc = train.drop(columns = {'letter'})
xtest_gbc = test.drop(columns = {'letter'})
ytrain_gbc = train['letter']
ytest_gbc = test['letter']

gbc = GradientBoostingClassifier(n_estimators=500, max_leaf_nodes=10, random_state=2023)
gbc.fit(xtrain_gbc, ytrain_gbc)
gbc_pred = gbc.predict(xtest_gbc)
model_2f_acc = sum(gbc_pred == ytest_gbc) / len(ytest_gbc)
print(f'GBC Test Accuracy: {model_2f_acc:.4f}')

GBC Test Accuracy: 0.9636
